In [1]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [2]:
data_path = "/Users/jaehyoyi/CR_finetune/CR_finetuning/Dataset/300_기타20개/CR_Human_Annotation_Train_300_Messages_Format.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:") 
for message in dataset[0]["messages"]:
    print(message)

Num examples: 300
First example:
{'role': 'system', 'content': 'You are an AI expert in Company reviews. You are asked to classify unlabeled sentences for 6 company labels. A text is classified into a company review category if the author is expressing the category in the text. ‘Company Review’ is divided into six categories, as follows: \n1. 성장 가능성 및 비전 : 회사의 장기적인 성장 가능성, 사업 확장, 업계 내 경쟁력, 비전, 기업의 미래 방향성, 개인의 성장가능성과 \n2. 비전 복지 및 급여: 직원들에게 제공되는 급여 수준, 보너스, 연봉 인상, 복지 혜택, 건강 보험, 연차 휴가, 사내교육 \n3. 근무 환경과 WLB : 근무 환경, 일과 삶의 균형, 업무 강도, 근무 시간, 재택근무 가능성, 휴식 시간, 사무실 시설, 근무지 위치, 조직 간 이동, 피로감 \n4. 사내문화: 회사 내 분위기, 사내정치(라인), 일하는 방식, 직원들 간의 관계, 소통 방식, 협업 방식, 연차 자유롭게 쓰는 분위기, 수평적, 수직적, 보고 문서 작업, 고용유지(엘무원)-제도 \n5. 경영진(리더십): 보상/성과평가, 인사 정책, 의사결정 방식, 경영 전략, 직원들에 대한 배려, 신입사원 채용, 직급/체계, 진급누락 적절한 타이밍: 경영진이 적재적소에 조직 구성을 해야하기 때문, 고용유지(엘무원)-제도 \n6. 기타 : 위에 것에 해당하지 않는 텍스트'}
{'role': 'user', 'content': '"대기업다운 시스템, 보상이 이루어지며, 금년도 7월을 목표로 WLB 개선을 위해 주 52시간 검토 및 적용 직전"'}
{'role': 'assistant', 'content': '성장 가능성 및 비

### Format Validation

In [3]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


## Token Counting Utilities

In [4]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [6]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

# Define or import num_tokens_from_messages and num_assistant_tokens_from_messages here

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)

# Define or import print_distribution here
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")

# Ensure proper number without commas
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")


Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 600, 950
mean / median: 643.3, 633.0
p5 / p95: 615.0, 676.2

#### Distribution of num_assistant_tokens_per_example:
min / max: 3, 36
mean / median: 13.61, 12.0
p5 / p95: 4.0, 24.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


### Cost Estimation

In [7]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~192990 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~578970 tokens
